In [1]:
import os
from google_play_scraper import Sort, reviews
import pandas as pd
from tqdm import tqdm
from langdetect import detect
from deep_translator import GoogleTranslator
from transformers import pipeline
import os
import time

d:\kaimtenx\week22\bank-review-analysis\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
# Create data folder
os.makedirs("../data", exist_ok=True)

# Updated app IDs
bank_ids = {
    "cbe": "com.combanketh.mobilebanking",
    "absiniya": "com.boa.boaMobileBanking",
    "dashin": "com.dashen.dashensuperapp"
}

In [3]:
# Filter function: keep only Amharic or English
def is_am_or_en(text):
    try:
        lang = detect(text)
        return lang in ['am', 'en']
    except:
        return False

# Scraper function
def scrape_400_reviews(app_id, bank_name, target_count=450):
    print(f"\n🔍 Scraping {bank_name.upper()}...")

    all_reviews = []
    seen = set()
    token = None

    while len(all_reviews) < target_count:
        rvws, token = reviews(
            app_id,
            lang='en',
            country='et',
            sort=Sort.NEWEST,
            count=100,
            continuation_token=token
        )

        if not rvws:
            print(" No more reviews available.")
            break

        for r in rvws:
            content = r['content']
            if content in seen or not is_am_or_en(content):
                continue

            seen.add(content)
            all_reviews.append({
                'review': content,
                'rating': r['score'],
                'date': r['at'],
                'bank': bank_name.capitalize(),
                'source': 'Google Play'
            })

            if len(all_reviews) >= target_count:
                break

        print(f"✅ {bank_name}: {len(all_reviews)} reviews collected")

        if token is None:
            break

        time.sleep(1)  # polite pause

    df = pd.DataFrame(all_reviews)
    df.to_csv(f"../data/{bank_name}.csv", index=False)
    print(f"💾 Saved to ../data/{bank_name}.csv")
    return df
# Run for all banks
for bank, app_id in bank_ids.items():
    scrape_400_reviews(app_id, bank)


🔍 Scraping CBE...
✅ cbe: 43 reviews collected
✅ cbe: 98 reviews collected
✅ cbe: 151 reviews collected
✅ cbe: 201 reviews collected
✅ cbe: 247 reviews collected
✅ cbe: 300 reviews collected
✅ cbe: 355 reviews collected
✅ cbe: 411 reviews collected
✅ cbe: 450 reviews collected
💾 Saved to ../data/cbe.csv

🔍 Scraping ABSINIYA...
✅ absiniya: 64 reviews collected
✅ absiniya: 126 reviews collected
✅ absiniya: 186 reviews collected
✅ absiniya: 241 reviews collected
✅ absiniya: 296 reviews collected
✅ absiniya: 357 reviews collected
✅ absiniya: 431 reviews collected
✅ absiniya: 450 reviews collected
💾 Saved to ../data/absiniya.csv

🔍 Scraping DASHIN...
✅ dashin: 54 reviews collected
✅ dashin: 148 reviews collected
✅ dashin: 214 reviews collected
✅ dashin: 277 reviews collected
✅ dashin: 314 reviews collected
 No more reviews available.
💾 Saved to ../data/dashin.csv


In [4]:
# Step 1: Load scraped CSVs
cbe = pd.read_csv("../data/cbe.csv")
boa = pd.read_csv("../data/absiniya.csv")
dashin = pd.read_csv("../data/dashin.csv")
# Step 2: Combine into one DataFrame
df = pd.concat([cbe, boa, dashin], ignore_index=True)

# Step 3: Ensure required columns
df = df[['review', 'rating', 'date', 'bank', 'source']]

# Step 4: Print overall missing values
print("🔍 Total missing values (all banks):")
print(df.isnull().sum())

# Step 5: Print overall duplicates
total_dups = df.duplicated(subset=['review', 'bank']).sum()
print(f"\n🔁 Total duplicate reviews (by review + bank): {total_dups}")

# Step 6: Drop rows where 'review' is missing or empty
before = df.shape[0]
df = df[df['review'].notna()]
df = df[df['review'].str.strip() != '']
after = df.shape[0]
print(f"\n Dropped {before - after} rows with missing/empty review.")

# Step 7: Remove duplicates per bank
df = df.drop_duplicates(subset=['review', 'bank'])

# Step 8: Final counts per bank
print("\n Final review count per bank after cleaning:")
print(df['bank'].value_counts())

# Step 9: Save cleaned data
df.to_csv("../data/BANKS_review.csv", index=False)
print("\n Cleaned data saved to: ../data/BANKS_review.csv")

🔍 Total missing values (all banks):
review    0
rating    0
date      0
bank      0
source    0
dtype: int64

🔁 Total duplicate reviews (by review + bank): 0

 Dropped 0 rows with missing/empty review.

 Final review count per bank after cleaning:
bank
Cbe         450
Absiniya    450
Dashin      314
Name: count, dtype: int64

 Cleaned data saved to: ../data/BANKS_review.csv


In [5]:
# language detection and translation into english
from langdetect import detect
from deep_translator import GoogleTranslator

In [6]:
# Step 1: Load cleaned reviews
df = pd.read_csv("../data/BANKS_review.csv")

# Step 2: Translation function
def translate_to_english(text):
    try:
        lang = detect(text)
        if lang == 'am':
            translated = GoogleTranslator(source='auto', target='en').translate(text)
            return translated
        elif lang == 'en':
            return text  # Already in English
        else:
            return "UNTRANSLATED"
    except Exception as e:
        return "UNTRANSLATED"

# Step 3: Translate Amharic reviews only
print("🔁 Translating reviews (this may take several minutes)...")
df['translated_review'] = df['review'].apply(translate_to_english)

# Step 4: Save translated dataset
df.to_csv("../data/BANKS_review_translated.csv", index=False)
print("✅ Translated reviews saved to: ../data/BANKS_review_translated.csv")

🔁 Translating reviews (this may take several minutes)...
✅ Translated reviews saved to: ../data/BANKS_review_translated.csv


In [8]:
from tqdm import tqdm
tqdm.pandas()
def detect_language(text):
    try:
        return detect(str(text))
    except:
        return 'unknown'

df['lang_detected'] = df['translated_review'].progress_apply(detect_language)
df = df[df['lang_detected'] == 'en']
df.to_csv("../data/BANKS_review_translatedENGLISH.csv", index=False)


100%|██████████| 1182/1182 [00:03<00:00, 320.11it/s]


In [9]:


from langdetect.lang_detect_exception import LangDetectException
from collections import Counter

In [10]:
# Enable progress bar for pandas
tqdm.pandas()

# Load your dataset
df = pd.read_csv("../data/BANKS_review_translatedENGLISH.csv")  # or your translated file

# Language detection function
def detect_language(text):
    try:
        return detect(str(text))
    except LangDetectException:
        return 'unknown'

# Detect language of each translated review
print("🔍 Detecting languages in translated reviews...")
df['lang_detected'] = df['translated_review'].progress_apply(detect_language)

# Count each language
lang_counts = Counter(df['lang_detected'])

# Print result
print("\n🌐 Language Distribution in translated_review:")
for lang, count in lang_counts.items():
    print(f"{lang}: {count} reviews")

🔍 Detecting languages in translated reviews...


100%|██████████| 1175/1175 [00:03<00:00, 306.96it/s]


🌐 Language Distribution in translated_review:
en: 1170 reviews
af: 3 reviews
so: 1 reviews
de: 1 reviews


In [12]:
df = pd.read_csv("../data/BANKS_review_translatedENGLISH.csv")  # or your translated file# Keep only rows where language is English
english_only_df = df[df['lang_detected'] == 'en']

# Save cleaned data
english_only_df.to_csv("../data/translated_allbanksdata_cleaned.csv", index=False)
print(f" Cleaned dataset saved to: ../data/translated_allbanksdata_cleaned.csv")
print(f"Remaining English reviews: {english_only_df.shape[0]}")

 Cleaned dataset saved to: ../data/translated_allbanksdata_cleaned.csv
Remaining English reviews: 1175
